# Code adapted from https://github.com/scjones5/nyt-bestsellers

In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 100)

In [2]:
r = requests.get("https://api.nytimes.com/svc/books/v3/lists/2019-08-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH")

In [3]:
bs = r.json()

In [4]:
for key in bs.keys():
    print(key)

status
copyright
num_results
last_modified
results


In [5]:
bs = json_normalize(bs["results"]["books"])

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_72299/577837607.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  bs = json_normalize(bs["results"]["books"])


In [7]:
#get list names
#get other lists
r2 = requests.get('https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH')

In [8]:
bs2 = r2.json()

In [9]:
print(bs2)

{'status': 'OK', 'copyright': 'Copyright (c) 2022 The New York Times Company.  All Rights Reserved.', 'num_results': 59, 'results': [{'list_name': 'Combined Print and E-Book Fiction', 'display_name': 'Combined Print & E-Book Fiction', 'list_name_encoded': 'combined-print-and-e-book-fiction', 'oldest_published_date': '2011-02-13', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Combined Print and E-Book Nonfiction', 'display_name': 'Combined Print & E-Book Nonfiction', 'list_name_encoded': 'combined-print-and-e-book-nonfiction', 'oldest_published_date': '2011-02-13', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Hardcover Fiction', 'display_name': 'Hardcover Fiction', 'list_name_encoded': 'hardcover-fiction', 'oldest_published_date': '2008-06-08', 'newest_published_date': '2022-06-19', 'updated': 'WEEKLY'}, {'list_name': 'Hardcover Nonfiction', 'display_name': 'Hardcover Nonfiction', 'list_name_encoded': 'hardcover-nonfiction', '

In [ ]:
#it looks like only two young adult lists are still updated today:
    #Young Adult Paperback Monthly 'young-adult-paperback-monthly' (first published 2019-10-13)
        #call: 140 weeks (hopefully weekly code will work with monthly list?)
    #Young Adult Hardcover [Weekly] 'young-adult-hardcover' (first published 2015-08-30) 
        #call: 355 weeks

In [6]:
def parse_books(book_list, date):
    '''
    Based on: function parse_articles from
    This function takes in a response to the NYT books api and parses
    the books into a list of dictionaries
    '''
    books = []
    for i in book_list['results']['books']:
        dic = {}
        dic['age_group'] = i['age_group']
        dic['author'] = i['author'].encode("utf8")
        dic['book_review_link'] = i['book_review_link'].encode("utf8")
        dic['contributor'] = i['contributor'].encode("utf8")
        dic['dagger'] = i['dagger']
        dic['first_chapter_link'] = i['first_chapter_link'].encode("utf8")
        dic['price'] = i['price']
        dic['primary_isbn10'] = i['primary_isbn10'].encode("utf8")
        dic['primary_isbn13'] = i['primary_isbn13'].encode("utf8")
        dic['publisher'] = i['publisher'].encode("utf8")
        dic['sunday_review_link'] = i['sunday_review_link'].encode("utf8")
        dic['title'] = i['title'].encode("utf8")
        dic['weeks_on_list'] = i['weeks_on_list']
        dic['date'] = date  # add a date so that we know when it was a best-seller
        books.append(dic)
    return(books)


## start with Young Adult Paperback Monthly 'young-adult-paperback-monthly'

In [19]:
#sample request: https://api.nytimes.com/svc/books/v3/lists/2017-03-13/Combined%20Print%20and%20E-Book%20Fiction.json?api_key=<your_api_key>
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "young-adult-paperback-monthly"   #you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH"


monthly_paperback_bestsellers = []
weeks_monthly = []
num_calls_to_make = 140 # you can use up to 1000 api calls a day so max=1000
ctr = 0

# lists update once a week, every Sunday
for i in range(0,num_calls_to_make):
    today = datetime.datetime.today()
    idx = (today.weekday() + 1) % 7
    last_sun = today - datetime.timedelta((7*ctr)+idx)
    last_sun = last_sun.date()
    print(last_sun)
    url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, last_sun, LIST, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    if (r.status_code == 200):
        results = r.json()
        books = json_normalize(results["results"]["books"]) #parse_books(results, date)
        monthly_paperback_bestsellers.append(books)
        weeks_monthly.append(str(last_sun))
    else:
        print("API call did not work!")
        print(r)
        #break
    ctr = ctr + 1
    time.sleep(6)
#     if (bestsellers[0].keys() is not None):
#        keys = bestsellers[0].keys()
#        # Print all results to a csv file
#        with open('bestsellers.csv', 'wb') as output_file:
#            dict_writer = csv.DictWriter(output_file, keys)
#            dict_writer.writeheader()
#            dict_writer.writerows(bestsellers)

2022-06-12
https://api.nytimes.com/svc/books/v3/lists/2022-06-12/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH


/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_72299/398506677.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  books = json_normalize(results["results"]["books"]) #parse_books(results, date)


2022-06-05
https://api.nytimes.com/svc/books/v3/lists/2022-06-05/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-29
https://api.nytimes.com/svc/books/v3/lists/2022-05-29/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-22
https://api.nytimes.com/svc/books/v3/lists/2022-05-22/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-15
https://api.nytimes.com/svc/books/v3/lists/2022-05-15/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-08
https://api.nytimes.com/svc/books/v3/lists/2022-05-08/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-01
https://api.nytimes.com/svc/books/v3/lists/2022-05-01/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-24
https://api.nytimes.com/svc/books/v3/lists/2022-04-24/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-17
https://api.nytimes.com/svc/books/v3/

2021-04-04
https://api.nytimes.com/svc/books/v3/lists/2021-04-04/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-28
https://api.nytimes.com/svc/books/v3/lists/2021-03-28/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-21
https://api.nytimes.com/svc/books/v3/lists/2021-03-21/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-14
https://api.nytimes.com/svc/books/v3/lists/2021-03-14/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-07
https://api.nytimes.com/svc/books/v3/lists/2021-03-07/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-28
https://api.nytimes.com/svc/books/v3/lists/2021-02-28/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-21
https://api.nytimes.com/svc/books/v3/lists/2021-02-21/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-14
https://api.nytimes.com/svc/books/v3/

2020-02-02
https://api.nytimes.com/svc/books/v3/lists/2020-02-02/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-26
https://api.nytimes.com/svc/books/v3/lists/2020-01-26/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-19
https://api.nytimes.com/svc/books/v3/lists/2020-01-19/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-12
https://api.nytimes.com/svc/books/v3/lists/2020-01-12/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-05
https://api.nytimes.com/svc/books/v3/lists/2020-01-05/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2019-12-29
https://api.nytimes.com/svc/books/v3/lists/2019-12-29/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2019-12-22
https://api.nytimes.com/svc/books/v3/lists/2019-12-22/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2019-12-15
https://api.nytimes.com/svc/books/v3/

In [20]:
monthly_paperback_bestsellers_df = pd.concat(monthly_paperback_bestsellers)
monthly_paperback_bestsellers_df.head(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,0,0,0,0062457802,9780062457806,Quill Tree,,0.00,THEY BOTH DIE AT THE END,Adam Silvera,by Adam Silvera,,https://storage.googleapis.com/du-prd/books/im...,327,495,https://www.amazon.com/They-Both-Die-at-End/dp...,,,,,,"[{'isbn10': '0062457799', 'isbn13': '978006245...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/ee569b8a-1b59-5af7-ab27-c6e697978460
1,2,0,0,0,0,0385741278,9780385741279,Ember,,0.00,WE WERE LIARS,E. Lockhart,by E. Lockhart,,https://storage.googleapis.com/du-prd/books/im...,128,194,http://www.amazon.com/We-Were-Liars-E-Lockhart...,,,,https://www.nytimes.com/2014/05/11/books/revie...,,"[{'isbn10': '0375984402', 'isbn13': '978037598...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/a474c5c9-ee75-5774-bda1-74f7bc37f83b
2,3,0,0,0,0,0759555400,9780759555402,"Little, Brown",,0.00,THE INHERITANCE GAMES,Jennifer Lynn Barnes,by Jennifer Lynn Barnes,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/1368052401?tag=NYTBS...,,,,,,"[{'isbn10': '1368052401', 'isbn13': '978136805...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/e2abd326-9377-54d9-bf9c-5125478dd1c1
3,4,0,0,0,0,1101934743,9781101934746,Ember,,0.00,GIRL IN PIECES,Kathleen Glasgow,by Kathleen Glasgow,,https://storage.googleapis.com/du-prd/books/im...,328,495,http://www.amazon.com/Girl-Pieces-Kathleen-Gla...,,,,,,"[{'isbn10': '1101934719', 'isbn13': '978110193...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/4c61a5f1-f0f8-5184-978c-c7056c96a644
4,5,0,0,0,0,1536225266,9781536225266,Walker,,0.00,ALL OUR HIDDEN GIFTS,Caroline O'Donoghue,by Caroline O'Donoghue,,https://storage.googleapis.com/du-prd/books/im...,333,500,https://www.amazon.com/dp/1536225266?tag=NYTBS...,,,,,,"[{'isbn10': '1536225266', 'isbn13': '978153622...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/551bad5f-4a32-5a4c-9759-87a01880bd8d
5,6,0,0,0,0,0375842209,9780375842207,Knopf,,0.00,THE BOOK THIEF,Markus Zusak,by Markus Zusak,,https://storage.googleapis.com/du-prd/books/im...,321,495,http://www.amazon.com/The-Book-Thief-Markus-Zu...,,https://www.nytimes.com/2006/03/27/books/27mas...,,https://www.nytimes.com/2006/05/14/books/revie...,,"[{'isbn10': '0375842209', 'isbn13': '978037584...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/535c6937-87ea-5803-a820-cc3cb3114056
6,7,0,0,0,0,0593125010,9780593125014,Delacorte,,0.00,THE FEAR,Natasha Preston,by Natasha Preston,,https://storage.googleapis.com/du-prd/books/im...,333,500,https://www.amazon.com/dp/0593125010?tag=NYTBS...,,,,,,"[{'isbn10': '0593125010', 'isbn13': '978059312...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/2955d090-51ef-5da8-b9b8-0af2400b248c
7,8,0,0,0,0,125007696X,9781250076960,Square Fish,,0.00,SIX OF CROWS,Leigh Bardugo,by Leigh Bardugo,,https://storage.googleapis.com/du-prd/books/im...,330,491,http://www.amazon.com/Six-Crows-Leigh-Bardugo/...,,,,,,"[{'isbn10': '1627792120', 'isbn13': '978162779...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/90141a33-530f-5f44-a4b9-90ef74230800
8,9,0,0,0,0,0525555277,9780525555278,Dutton,,0.00,LAST NIGHT AT THE TELEGRAPH CLUB,Malinda Lo,by Malinda Lo,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/0525555277?tag=NYTBS...,,,,,,"[{'isbn10': '0525555277', 'isbn13': '978052555...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/af17ecaf-8647-5744-8ab5-8343f50a39bb
9,10,0,0,0,0,1728231892,9781728231891,Sourcebooks Fire,,0.00,LOCK THE DOORS,Vincent Ralph,by Vincent Ralph,,https://storage.googleapis.com/du-prd/books/im...,333,500,https://www.amazon.com/dp/1728

In [21]:
monthly_paperback_bestsellers_df.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,0,0,0,,9780593202784,Penguin,,0.00,SALT TO THE SEA,Ruta Sepetys,by Ruta Sepetys,,https://storage.googleapis.com/du-prd/books/im...,319,495,http://www.amazon.com/Salt-Sea-Ruta-Sepetys/dp...,,,,https://www.nytimes.com/2016/02/14/books/revie...,,"[{'isbn10': '0399160302', 'isbn13': '978039916...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/3279e899-1bd9-5550-9177-a712918b0a8f
1,2,4,0,0,0,0316013692,9780316013697,"Little, Brown",,0.00,THE ABSOLUTELY TRUE DIARY OF A PART-TIME INDIAN,Sherman Alexie.,by Sherman Alexie. Illustrated by Ellen Forney,Illustrated by Ellen Forney,https://storage.googleapis.com/du-prd/books/im...,322,495,http://www.amazon.com/Absolutely-True-Diary-Pa...,,,,https://www.nytimes.com/2007/11/11/books/revie...,,"[{'isbn10': '0316013692', 'isbn13': '978031601...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/dbe607f5-6f94-5709-b06b-42087923b34f
2,3,0,0,0,0,1481438263,9781481438261,Atheneum/Caitlyn Dlouhy,,0.00,LONG WAY DOWN,Jason Reynolds,by Jason Reynolds,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Long-Way-Down-Jason-Rey...,,,,,,"[{'isbn10': '1481438255', 'isbn13': '978148143...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/434525fe-4696-5f1c-a6d3-de2fbd14c542
3,4,0,0,0,0,144247243X,9781442472433,Simon & Schuster,,0.00,SCYTHE,Neal Shusterman,by Neal Shusterman,,https://storage.googleapis.com/du-prd/books/im...,327,495,https://www.amazon.com/Scythe-Arc-Neal-Shuster...,,,,,,[],"[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/63197ac6-9859-59d8-b289-8bca3248f3d4
4,5,0,0,0,0,1101939524,9781101939529,Ember,,0.00,DEAR MARTIN,Nic Stone,by Nic Stone,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Dear-Martin-Nic-Stone/d...,,,,,,"[{'isbn10': '1101939494', 'isbn13': '978110193...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/0a9f4692-2d3f-5471-a2b3-0e09aa776f1e
5,6,0,0,0,0,0525555374,9780525555377,Penguin,,0.00,TURTLES ALL THE WAY DOWN,John Green,by John Green,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Turtles-All-Down-John-G...,,,,,,"[{'isbn10': '0525555366', 'isbn13': '978052555...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/96192ed4-5540-5910-a127-34966b7fa730
6,7,0,0,0,0,0553496719,9780553496710,Ember,,0.00,THE SUN IS ALSO A STAR,Nicola Yoon,by Nicola Yoon,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Sun-Also-Star-Nicola-Yo...,,,,,,"[{'isbn10': '0553496689', 'isbn13': '978055349...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/c3732e65-eaa5-5c5e-b09f-99357e1d4eb8
7,8,0,0,0,0,1481463349,9781481463348,Atheneum/Caitlyn Dlouhy,,0.00,ALL AMERICAN BOYS,Jason Reynolds and Brendan Kiely,by Jason Reynolds and Brendan Kiely,,https://storage.googleapis.com/du-prd/books/im...,330,494,http://www.amazon.com/All-American-Boys-Jason-...,,,,,,"[{'isbn10': '1481463330', 'isbn13': '978148146...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/69ce4b50-86bd-5d0d-8a3f-9c3c7c79df3b
8,9,0,0,0,0,0385751532,9780385751537,Random House,,0.00,THE BOY IN THE STRIPED PAJAMAS,John Boyne,by John Boyne,,https://storage.googleapis.com/du-prd/books/im...,321,495,http://www.amazon.com/Striped-Pajamas-Young-Re...,,,,https://www.nytimes.com/2006/11/12/books/Scott...,,"[{'isbn10': '0385751532', 'isbn13': '978038575...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f4a17e7a-ad4b-5b17-a24c-6cacd9ac2219
9,10,0,0,0,0,0553496670,9780553496673,Ember,,0.00,"EVERYTHING, EVERYTHING",Nicola Yoon,by Nicola Yoon,,https://storage.googleapis.co

In [22]:
#repeat weeks 10 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 10
repeated_weeks_monthly = []
for element in weeks_monthly:
    for i in range(k):
       repeated_weeks_monthly.append(element)

In [23]:
#check work
len(repeated_weeks_monthly)

1400

In [24]:
#check work
len(monthly_paperback_bestsellers_df)

1400

In [25]:
#let's go ahead and add a week column
monthly_paperback_bestsellers_df['week'] = repeated_weeks_monthly

#and check to make sure it worked correctly
monthly_paperback_bestsellers_df.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week
0,1,0,0,0,0,,9780593202784,Penguin,,0.00,SALT TO THE SEA,Ruta Sepetys,by Ruta Sepetys,,https://storage.googleapis.com/du-prd/books/im...,319,495,http://www.amazon.com/Salt-Sea-Ruta-Sepetys/dp...,,,,https://www.nytimes.com/2016/02/14/books/revie...,,"[{'isbn10': '0399160302', 'isbn13': '978039916...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/3279e899-1bd9-5550-9177-a712918b0a8f,2019-10-20
1,2,4,0,0,0,0316013692,9780316013697,"Little, Brown",,0.00,THE ABSOLUTELY TRUE DIARY OF A PART-TIME INDIAN,Sherman Alexie.,by Sherman Alexie. Illustrated by Ellen Forney,Illustrated by Ellen Forney,https://storage.googleapis.com/du-prd/books/im...,322,495,http://www.amazon.com/Absolutely-True-Diary-Pa...,,,,https://www.nytimes.com/2007/11/11/books/revie...,,"[{'isbn10': '0316013692', 'isbn13': '978031601...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/dbe607f5-6f94-5709-b06b-42087923b34f,2019-10-20
2,3,0,0,0,0,1481438263,9781481438261,Atheneum/Caitlyn Dlouhy,,0.00,LONG WAY DOWN,Jason Reynolds,by Jason Reynolds,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Long-Way-Down-Jason-Rey...,,,,,,"[{'isbn10': '1481438255', 'isbn13': '978148143...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/434525fe-4696-5f1c-a6d3-de2fbd14c542,2019-10-20
3,4,0,0,0,0,144247243X,9781442472433,Simon & Schuster,,0.00,SCYTHE,Neal Shusterman,by Neal Shusterman,,https://storage.googleapis.com/du-prd/books/im...,327,495,https://www.amazon.com/Scythe-Arc-Neal-Shuster...,,,,,,[],"[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/63197ac6-9859-59d8-b289-8bca3248f3d4,2019-10-20
4,5,0,0,0,0,1101939524,9781101939529,Ember,,0.00,DEAR MARTIN,Nic Stone,by Nic Stone,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Dear-Martin-Nic-Stone/d...,,,,,,"[{'isbn10': '1101939494', 'isbn13': '978110193...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/0a9f4692-2d3f-5471-a2b3-0e09aa776f1e,2019-10-20
5,6,0,0,0,0,0525555374,9780525555377,Penguin,,0.00,TURTLES ALL THE WAY DOWN,John Green,by John Green,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Turtles-All-Down-John-G...,,,,,,"[{'isbn10': '0525555366', 'isbn13': '978052555...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/96192ed4-5540-5910-a127-34966b7fa730,2019-10-20
6,7,0,0,0,0,0553496719,9780553496710,Ember,,0.00,THE SUN IS ALSO A STAR,Nicola Yoon,by Nicola Yoon,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Sun-Also-Star-Nicola-Yo...,,,,,,"[{'isbn10': '0553496689', 'isbn13': '978055349...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/c3732e65-eaa5-5c5e-b09f-99357e1d4eb8,2019-10-20
7,8,0,0,0,0,1481463349,9781481463348,Atheneum/Caitlyn Dlouhy,,0.00,ALL AMERICAN BOYS,Jason Reynolds and Brendan Kiely,by Jason Reynolds and Brendan Kiely,,https://storage.googleapis.com/du-prd/books/im...,330,494,http://www.amazon.com/All-American-Boys-Jason-...,,,,,,"[{'isbn10': '1481463330', 'isbn13': '978148146...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/69ce4b50-86bd-5d0d-8a3f-9c3c7c79df3b,2019-10-20
8,9,0,0,0,0,0385751532,9780385751537,Random House,,0.00,THE BOY IN THE STRIPED PAJAMAS,John Boyne,by John Boyne,,https://storage.googleapis.com/du-prd/books/im...,321,495,http://www.amazon.com/Striped-Pajamas-Young-Re...,,,,https://www.nytimes.com/2006/11/12/books/Scott...,,"[{'isbn10': '0385751532', 'isbn13': '978038575...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/f4a17e7a-ad4b-5b17-a24c-6cacd9ac2219,2019-10-20
9,10,0,0,0,0,0553496670,9780

In [26]:
#and save to csv
monthly_paperback_bestsellers_df.to_csv("../data/monthly_paperback_YA_bestsellers_since_oct_2019.csv")

## repeat for Young Adult Hardcover [Weekly] 'young-adult-hardcover'

In [ ]:
#sample request: https://api.nytimes.com/svc/books/v3/lists/2017-03-13/Combined%20Print%20and%20E-Book%20Fiction.json?api_key=<your_api_key>
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "young-adult-paperback-monthly"   #you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH"


weekly_hardcover_bestsellers = []
weeks = []
num_calls_to_make = 355 # you can use up to 1000 api calls a day so max=1000
ctr = 0

# lists update once a week, every Sunday
for i in range(0,num_calls_to_make):
    today = datetime.datetime.today()
    idx = (today.weekday() + 1) % 7
    last_sun = today - datetime.timedelta((7*ctr)+idx)
    last_sun = last_sun.date()
    print(last_sun)
    url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, last_sun, LIST, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    if (r.status_code == 200):
        results = r.json()
        books = json_normalize(results["results"]["books"]) #parse_books(results, date)
        weekly_hardcover_bestsellers.append(books)
        weeks.append(str(last_sun))
    else:
        print("API call did not work!")
        print(r)
        #break
    ctr = ctr + 1
    time.sleep(6)
#     if (bestsellers[0].keys() is not None):
#        keys = bestsellers[0].keys()
#        # Print all results to a csv file
#        with open('bestsellers.csv', 'wb') as output_file:
#            dict_writer = csv.DictWriter(output_file, keys)
#            dict_writer.writeheader()
#            dict_writer.writerows(bestsellers)

2022-06-12
https://api.nytimes.com/svc/books/v3/lists/2022-06-12/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH


/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_72299/2816999393.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  books = json_normalize(results["results"]["books"]) #parse_books(results, date)


2022-06-05
https://api.nytimes.com/svc/books/v3/lists/2022-06-05/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-29
https://api.nytimes.com/svc/books/v3/lists/2022-05-29/young-adult-paperback-monthly?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH


In [ ]:
weekly_hardcover_bestsellers_df = pd.concat(weekly_hardcover_bestsellers)
weekly_hardcover_bestsellers_df.head(20)